## **協同過濾**

In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
# 逐行讀取數據集
content = []
with open(r"C:/yushan_race/需預測類別_交易次數合併資料_1211.csv") as fp:  
    content = fp.readlines()

In [ ]:
# 將用户、購買種類、和購買次數寫入字典data
data = {}
for line in content[1:(len(content)-1)]:
    line = line.strip().split(',')
    #如果字典中没有某位用户，则使用用户ID来创建这位用户
    if not line[0] in data.keys():
        data[line[0]] = {line[1]:line[2]}
    #否则直接添加以该用户ID为key字典中
    else:
        data[line[0]][line[1]] = line[2]

#{chid: {shop_tag1:txn_cnt1, shop_tag2:txn_cnt2} }
#data['10000000']

In [ ]:
from math import *
def Euclidean(user1,user2):
    #取出两位用户评论过的电影和评分
    user1_data=data[user1]
    user2_data=data[user2]
    distance = 0
    #找到两位用户都评论过的电影，并计算欧式距离
    for key in user1_data.keys():
        if key in user2_data.keys():
            #注意，distance越大表示两者越相似
            distance += pow(float(user1_data[key])-float(user2_data[key]),2)
 
    return 1/(1+sqrt(distance))#这里返回值越小，相似度越大
 
#计算某个用户与其他用户的相似度
def top10_simliar(userID):
    res = []
    for userid in data.keys():
        #排除与自己计算相似度
        if not userid == userID:
            simliar = Euclidean(userID,userid)
            res.append((userid,simliar))
    res.sort(key=lambda val:val[1])
    return res[:4] #推薦四個
 
#RES = top10_simliar('10000500')
#print(RES)

In [ ]:
def recommend(user):
    #相似度最高的用户
    top_sim_user = top10_simliar(user)[0][0]
    #相似度最高的用户的观影记录
    items = data[top_sim_user]
    recommendations = []
    #筛选出该用户未观看的电影并添加到列表中
    for item in items.keys():
        if item not in data[user].keys():
            recommendations.append((item,items[item]))
    recommendations.sort(key=lambda val:val[1],reverse=True)#按照评分排序
    #返回评分最高的10部电影
    return recommendations[:10]
 
#Recommendations = recommend('10229334')
#print(Recommendations)

## **推薦未滿三個類別需補上**

### **試試看填補邏輯 :**
1.   先計算大眾化類別跟系統推薦類別的差異(用大眾化-系統推薦)
2.   因為用set取差集順序可能會亂掉，所以按照原本的大眾化順序排序
3.   再把2.結果直接加到推薦系統的list裡面
4.   因為用後進先出去取前3名類別，所以設立while迴圈，如果推薦結果大於3就pop掉最新的元素，也就是最後推薦的類別



In [ ]:
pop_shop_tag=['6','10','12']   #大眾化的推薦名單，後進先出，所以順序不變
myid=list(data.keys())    #id
len(myid) #498040，代表有顧客沒有消費資料。所以推薦大眾化名單給他

498040

### 主程式

In [ ]:
%%time
myoutput=pd.DataFrame()
for i in range(0,10):
    result=[myid[i]]     
    tem=recommend(result[0])
    if len(tem) == 0:
        tem = tem
    else:
        tem = np.array(tem)
        tem = list(tem[0:len(tem),0])
        #tem = list(map(int, tem))
    c=list(set(pop_shop_tag)-set(tem))
    c.sort(key=pop_shop_tag.index)
    tem.extend(c)
    #while len(tem)>3:
    #    tem.pop()
    
    result.extend(tem[0:3])
    result=pd.DataFrame(result).T
    myoutput=pd.concat([myoutput,result],axis=0)
    #print(i)

Wall time: 22.3 s


In [ ]:
myoutput

,0,1,2,3
0,10000000,36,18,6
0,10000001,6,10,12
0,10000002,36,18,15
0,10000003,36,18,15
0,10000004,18,6,10
0,10000005,18,15,48
0,10000006,36,18,48
0,10000007,36,18,15
0,10000008,36,48,6
0,10000009,36,18,15
